In [1]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Risbha\Downloads\archive\criteo-uplift-v2.1.csv')
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,12.616365,10.059654,8.976429,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
1,12.616365,10.059654,9.002689,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
2,12.616365,10.059654,8.964775,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
3,12.616365,10.059654,9.002801,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
4,12.616365,10.059654,9.037999,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13979587,26.297764,10.059654,9.006250,4.679882,10.280525,4.115453,-3.282109,4.833815,3.839578,13.190056,5.300375,-0.168679,1,0,0,0
13979588,12.642207,10.679513,8.214383,-1.700105,10.280525,3.013064,-13.955150,6.269026,3.971858,13.190056,5.300375,-0.168679,1,0,0,1
13979589,12.976557,10.059654,8.381868,0.842442,11.029584,4.115453,-8.281971,4.833815,3.779212,23.570168,6.169187,-0.168679,1,0,1,0
13979590,24.805064,10.059654,8.214383,4.679882,10.280525,4.115453,-1.288207,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13979592 entries, 0 to 13979591
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   f0          float64
 1   f1          float64
 2   f2          float64
 3   f3          float64
 4   f4          float64
 5   f5          float64
 6   f6          float64
 7   f7          float64
 8   f8          float64
 9   f9          float64
 10  f10         float64
 11  f11         float64
 12  treatment   int64  
 13  conversion  int64  
 14  visit       int64  
 15  exposure    int64  
dtypes: float64(12), int64(4)
memory usage: 1.7 GB


In [3]:
df.isnull().sum()

f0            0
f1            0
f2            0
f3            0
f4            0
f5            0
f6            0
f7            0
f8            0
f9            0
f10           0
f11           0
treatment     0
conversion    0
visit         0
exposure      0
dtype: int64

In [4]:
print(df['treatment'].value_counts())

treatment
1    11882655
0     2096937
Name: count, dtype: int64


In [5]:
print(df['conversion'].value_counts())

conversion
0    13938818
1       40774
Name: count, dtype: int64


In [8]:
print("Conversion rate overall:", df['conversion'].mean())

Conversion rate overall: 0.002916680257907384


In [10]:
df_sample = df.sample(frac=0.05, random_state=42)
print("Sample shape:", df_sample.shape)

Sample shape: (698980, 16)


In [11]:
X = df_sample[[col for col in df_sample.columns if col.startswith('f')]]  # features f0–f11
T = df_sample['treatment']  # who saw the ad
y = df_sample['conversion']  # who converted (made a purchase)

In [12]:
print("Features shape:", X.shape)
print("Treatment unique:", T.unique())
print("Conversion rate:", y.mean())

Features shape: (698980, 12)
Treatment unique: [0 1]
Conversion rate: 0.0029628887807948727


In [13]:
from sklearn.model_selection import train_test_split

# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(
    X, y, T, test_size=0.2, random_state=42, stratify=T
)

print("Training data:", X_train.shape)
print("Testing data:", X_test.shape)

Training data: (559184, 12)
Testing data: (139796, 12)


In [14]:
from sklift.models import TwoModels
from sklearn.ensemble import RandomForestClassifier
from sklift.metrics import uplift_at_k, qini_auc_score

# uplift model
uplift_model = TwoModels(
    estimator_trmnt=RandomForestClassifier(n_estimators=50, random_state=42),
    estimator_ctrl=RandomForestClassifier(n_estimators=50, random_state=42),
    method='vanilla'
)

In [15]:
# Train the model
uplift_model.fit(X_train, y_train, T_train)

,estimator_trmnt,RandomForestC...ndom_state=42)
,estimator_ctrl,RandomForestC...ndom_state=42)
,method,'vanilla'
,n_estimators,50
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None


In [16]:
# Predict uplift on the test set
uplift_preds = uplift_model.predict(X_test)

In [17]:
# Qini AUC 
score = qini_auc_score(y_test, uplift_preds, T_test)
print("Qini AUC Score:", score)

Qini AUC Score: 0.11472266486719139


In [18]:

test_results = X_test.copy()
test_results['treatment'] = T_test.values
test_results['conversion'] = y_test.values
test_results['uplift_score'] = uplift_preds


top_customers = test_results.sort_values(by='uplift_score', ascending=False).head(10)
top_customers

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,uplift_score
10091787,12.616365,10.059654,8.414460,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,16.226044,5.300375,-0.168679,1,0,0.70
3820766,12.616365,10.059654,8.773658,4.679882,10.280525,4.115453,0.294443,4.833815,3.902645,13.190056,5.300375,-0.168679,1,0,0.66
5918923,12.616365,10.059654,8.312056,4.679882,11.029584,4.115453,0.294443,4.833815,3.955396,16.226044,5.437226,-0.267350,0,0,0.66
7701964,12.616365,10.059654,8.321288,4.679882,10.280525,4.115453,0.294443,4.833815,3.915574,23.570168,5.300375,-0.168679,1,0,0.66
1261212,12.616365,10.059654,8.318162,4.679882,10.280525,4.115453,0.294443,4.833815,3.927254,21.416100,5.300375,-0.168679,1,0,0.58
7825760,12.616365,10.059654,8.227373,4.679882,10.280525,4.115453,0.294443,4.833815,3.943716,18.380112,5.300375,-0.168679,1,0,0.56
8741120,12.616365,10.059654,8.227323,4.679882,10.280525,4.115453,0.294443,4.833815,3.943716,18.380112,5.300375,-0.168679,1,0,0.54
267037,12.616365,10.059654,8.227242,4.679882,10.280525,4.115453,0.294443,4.833815,3.943716,18.380112,5.300375,-0.168679,1,0,0.54
10010473,14.203017,10.059654,8.214383,1.114982,10.280525,4.115453,-9.404879,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0.50
7533991,12.616365,10.059654,8.715749,4.679882,10.280525,4.115453,0.294443,4.833815,3.920995,13.190056,5.300375,-0.168679,1,0,0.50


In [19]:
treated = df_sample[df_sample['treatment'] == 1]
control = df_sample[df_sample['treatment'] == 0]

treated_conv_rate = treated['conversion'].mean()
control_conv_rate = control['conversion'].mean()
lift = (treated_conv_rate - control_conv_rate) / control_conv_rate * 100

print(f"Treated Conversion Rate: {treated_conv_rate:.4f}")
print(f"Control Conversion Rate: {control_conv_rate:.4f}")
print(f"Conversion Lift: {lift:.2f}%")

Treated Conversion Rate: 0.0032
Control Conversion Rate: 0.0019
Conversion Lift: 69.77%


In [20]:
# Ad cost and conversion revenue assumptions
COST_PER_AD = 10
REVENUE_PER_CONVERSION = 500

# metrics per person
test_results['ad_cost'] = test_results['treatment'] * COST_PER_AD
test_results['revenue'] = test_results['conversion'] * REVENUE_PER_CONVERSION

# overall campaign summary
campaign_summary = {
    'total_ads_shown': test_results['treatment'].sum(),
    'total_conversions': test_results['conversion'].sum(),
    'total_cost': test_results['ad_cost'].sum(),
    'total_revenue': test_results['revenue'].sum()
}

campaign_summary['ROI'] = (
    (campaign_summary['total_revenue'] - campaign_summary['total_cost']) 
    / campaign_summary['total_cost']
)

print("📊 Campaign Summary:")
for k, v in campaign_summary.items():
    print(f"{k}: {v:,.2f}")

📊 Campaign Summary:
total_ads_shown: 118,827.00
total_conversions: 399.00
total_cost: 1,188,270.00
total_revenue: 199,500.00
ROI: -0.83


In [21]:
# uplift_score bins
test_results['uplift_segment'] = pd.cut(
    test_results['uplift_score'],
    bins=4,
    labels=['Low', 'Medium', 'High', 'Very High']
)

test_results['uplift_segment'].value_counts()

uplift_segment
Medium       139350
Low             235
High            198
Very High        13
Name: count, dtype: int64

In [22]:
uplift_summary = test_results.groupby('uplift_segment').agg({
    'conversion': 'mean',
    'ad_cost': 'mean',
    'revenue': 'mean',
    'uplift_score':'mean'
}).reset_index()

uplift_summary['ROI'] = (uplift_summary['revenue'] - uplift_summary['ad_cost']) / uplift_summary['ad_cost']

uplift_summary

C:\Users\Risbha\AppData\Local\Temp\ipykernel_1492\313529981.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  uplift_summary = test_results.groupby('uplift_segment').agg({


,uplift_segment,conversion,ad_cost,revenue,uplift_score,ROI
0,Low,0.106383,8.595745,53.191489,-0.139574,5.188119
1,Medium,0.002411,8.499390,1.205597,0.000983,-0.858155
2,High,0.186869,8.838384,93.434343,0.252525,9.571429
3,Very High,0.076923,8.461538,38.461538,0.563077,3.545455


In [23]:
import pandas as pd 

qini_score = score
overall_roi = campaign_summary['ROI']

kpi_data = {
    'MetricName': ['Qini_AUC_Score', 'Overall_Campaign_ROI'],
    'MetricValue': [qini_score, overall_roi]
}
kpi_df = pd.DataFrame(kpi_data)

print("KPI Table for Export:")
print(kpi_df)

KPI Table for Export:
             MetricName  MetricValue
0        Qini_AUC_Score     0.114723
1  Overall_Campaign_ROI    -0.832109


In [24]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

username = "root"
password = "*******"  
password_encoded = quote_plus(password)
host = "localhost"
port = 3306
database = "marketing_db"

# engine 
engine = create_engine(f"mysql+mysqlconnector://{username}:{password_encoded}@{host}:{port}/{database}")

In [25]:
# Exporting full campaign-level data
test_results.to_sql('campaign_results', con=engine, if_exists='replace', index=False)

# Exporting uplift summary table
uplift_summary.to_sql('uplift_summary', con=engine, if_exists='replace', index=False)

# Exporting the KPI table with the Qini Score and Overall ROI
kpi_df.to_sql('kpi_scores', con=engine, if_exists='replace', index=False)

print("✅ Tables exported to MySQL successfully!")

✅ Tables exported to MySQL successfully!


In [29]:
# ROI per segment from MySQL view
roi_df = pd.read_sql('SELECT * FROM ROI_per_segment', con=engine)
print(roi_df)

  uplift_segment  avg_conversion  avg_cost  avg_revenue       ROI
0         Medium          0.0024    8.4994       1.2056 -0.858155
1           High          0.1869    8.8384      93.4343  9.571429
2            Low          0.1064    8.5957      53.1915  5.188119
3      Very High          0.0769    8.4615      38.4615  3.545455
